# PDI Benchmark

In [2]:
import sys 
sys.path.append('../')
import pandas as pd
import numpy as np
import decouple
import db_funcs

In [3]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [27]:
# Helper functions
def convert_to_sql_strings(string_list):
    num_list = ['zero','one','two','three','four','five','six','seven','eight','nine']
    converted_strings = []
    for i in string_list:
        if '-' in i:
            i = i.replace('-','_')
        if i == '1inch':
            i = 'one_inch'
        converted_strings.append(i)
    return converted_strings

def convert_from_sql_strings(string_list):
    num_list = ['zero','one','two','three','four','five','six','seven','eight','nine']
    converted_strings = []
    for i in string_list:
        if '_' in i:
            i = i.replace('_','-')
        if i == 'one-inch':
            i = '1inch'
        converted_strings.append(i)
    return converted_strings

In [31]:
asset_list = ['uniswap', 'lido-dao', 'aave', 'rocket-pool', 'frax-share', 'havven', 'curve-dao-token', 'maker', 'convex-finance', '1inch', 'sushi', 'balancer', 'yearn-finance', 'ribbon-finance']
asset_list = convert_to_sql_strings(asset_list)
for asset in asset_list:
    db_funcs.does_column_exist(asset,'pdi_benchmark_data')

In [48]:
db_funcs.insert_values('2022-06-01',['uniswap','aave','lido_dao','compound_governance_token','yearn_finance','sushi','alchemix'],[0.300000,0.300000,0.120675,0.118707,0.081956,0.068245,0.010418],'pdi_benchmark_data')

In [46]:
db_funcs.drop_column('haaven','pdi_benchmark_data')